In [3]:
import sympy as sp

## Generators of SU(N)

In [4]:
def generators_su(N):
  matrices = []
  for i in range(N):
    for j in range(i+1, N):
      matrix = sp.zeros(N, N)
      matrix[i, j] = sp.Rational(1,2)
      matrix[j, i] = sp.Rational(1,2)
      matrices.append(matrix)
  for i in range(N):
    for j in range(i+1, N):
      matrix = sp.zeros(N, N)
      matrix[i, j] = -sp.I/2
      matrix[j, i] = sp.I/2
      matrices.append(matrix)
  for i in range(1,N):
    matrix = sp.zeros(N, N)
    for j in range(i):
      matrix[j, j] = 1
    matrix[i, i] = -i
    norm=sp.sqrt(2*sp.trace(matrix*matrix))
    matrix=norm**(-1)*matrix
    matrices.append(matrix)
  return matrices

In [5]:
generators_su(3)

[Matrix([
 [  0, 1/2, 0],
 [1/2,   0, 0],
 [  0,   0, 0]]),
 Matrix([
 [  0, 0, 1/2],
 [  0, 0,   0],
 [1/2, 0,   0]]),
 Matrix([
 [0,   0,   0],
 [0,   0, 1/2],
 [0, 1/2,   0]]),
 Matrix([
 [  0, -I/2, 0],
 [I/2,    0, 0],
 [  0,    0, 0]]),
 Matrix([
 [  0, 0, -I/2],
 [  0, 0,    0],
 [I/2, 0,    0]]),
 Matrix([
 [0,   0,    0],
 [0,   0, -I/2],
 [0, I/2,    0]]),
 Matrix([
 [1/2,    0, 0],
 [  0, -1/2, 0],
 [  0,    0, 0]]),
 Matrix([
 [sqrt(3)/6,         0,          0],
 [        0, sqrt(3)/6,          0],
 [        0,         0, -sqrt(3)/3]])]

Reordering

In [6]:
gell_mann_order = [0,3,6,1,4,2,5,7]
ordered_generators = [generators_su(3)[i] for i in gell_mann_order]

In [7]:
def T(a):
  return ordered_generators[a-1]

In [8]:
T(3)

Matrix([
[1/2,    0, 0],
[  0, -1/2, 0],
[  0,    0, 0]])

In [9]:
T(8)

Matrix([
[sqrt(3)/6,         0,          0],
[        0, sqrt(3)/6,          0],
[        0,         0, -sqrt(3)/3]])

## $su(3)$ *algebra*

In [10]:
def comm(a,b):
    return sp.simplify(a*b-b*a)

def acomm(a,b):
    return sp.simplify(a*b+b*a)

In [11]:
comm(T(1),T(4))==sp.I*T(7)/2

True

### Structure Constants

In [12]:
def f(a,b,c):
    return sp.simplify(-2*sp.I*sp.trace(comm(T(a),T(b))*T(c)))

In [13]:
# Print the nonzero structure constants
for a in range(1,9):
  for b in range(a,9):
    for c in range(b,9):
      if f(a,b,c)!= 0:
        print(f'f_{a}{b}{c}= {f(a,b,c)}')

f_123= 1
f_147= 1/2
f_156= -1/2
f_246= 1/2
f_257= 1/2
f_345= 1/2
f_367= -1/2
f_458= sqrt(3)/2
f_678= sqrt(3)/2


### Symmetric Constants

In [14]:
def d(a,b,c):
    return sp.simplify(2*sp.trace(acomm(T(a),T(b))*T(c)))

In [15]:
# Print the nonzero structure constants
for a in range(1,9):
  for b in range(a,9):
    for c in range(b,9):
      if d(a,b,c)!= 0:
        print(f'd_{a}{b}{c}= {d(a,b,c)}')

d_118= sqrt(3)/3
d_146= 1/2
d_157= 1/2
d_228= sqrt(3)/3
d_247= -1/2
d_256= 1/2
d_338= sqrt(3)/3
d_344= 1/2
d_355= 1/2
d_366= -1/2
d_377= -1/2
d_448= -sqrt(3)/6
d_558= -sqrt(3)/6
d_668= -sqrt(3)/6
d_778= -sqrt(3)/6
d_888= -sqrt(3)/3


## Adjoint Representation

In [16]:
def A(a):
    res=sp.zeros(8)
    for i in range(8):
        for j in range(8):
            res[i,j]=-sp.I*f(a,i+1,j+1)
    return sp.simplify(res)

In [17]:
A(3)

Matrix([
[0, -I, 0,   0,    0,    0,   0, 0],
[I,  0, 0,   0,    0,    0,   0, 0],
[0,  0, 0,   0,    0,    0,   0, 0],
[0,  0, 0,   0, -I/2,    0,   0, 0],
[0,  0, 0, I/2,    0,    0,   0, 0],
[0,  0, 0,   0,    0,    0, I/2, 0],
[0,  0, 0,   0,    0, -I/2,   0, 0],
[0,  0, 0,   0,    0,    0,   0, 0]])

Adjoint basis

In [18]:
def As(a):
  # Adjoint basis state |A_a>
  d=8
  return sp.eye(d).col(a-1)

In [19]:
A(1)*As(2)==sp.I*As(3)

True

In [20]:
A(1)*As(4)==(sp.I/2)*As(7)

True

Diagonalization, eigenvector normalization rephasing and reordering

In [21]:
O0,A3D=A(3).diagonalize()

In [22]:
def modal():
    O1=O0*sp.sqrt(sp.conjugate(sp.transpose(O0))*O0)**(-1)
    phase=sp.eye(8)
    for j in range(8):
        for i in range(8):
            if O1[i,j]!=0:
                phase[j,j]=sp.exp(-sp.I*sp.arg(O1[i,j]))
                break
    modal0=O1*phase
    res=sp.zeros(8)
    res[:,0]=modal0[:,3]
    res[:,1]=modal0[:,4]
    res[:,2]=modal0[:,7]
    res[:,3]=modal0[:,0]
    res[:,4]=modal0[:,5]
    res[:,5]=modal0[:,1]
    res[:,6]=modal0[:,2]
    res[:,7]=modal0[:,6]
    return res

In [23]:
modal()

Matrix([
[0, 0,   sqrt(2)/2,    sqrt(2)/2,           0,            0,           0,            0],
[0, 0, sqrt(2)*I/2, -sqrt(2)*I/2,           0,            0,           0,            0],
[1, 0,           0,            0,           0,            0,           0,            0],
[0, 0,           0,            0,   sqrt(2)/2,    sqrt(2)/2,           0,            0],
[0, 0,           0,            0, sqrt(2)*I/2, -sqrt(2)*I/2,           0,            0],
[0, 0,           0,            0,           0,            0,   sqrt(2)/2,    sqrt(2)/2],
[0, 0,           0,            0,           0,            0, sqrt(2)*I/2, -sqrt(2)*I/2],
[0, 1,           0,            0,           0,            0,           0,            0]])

Adjoint generators in the new basis

In [24]:
def Y(a):
  # Adjoint generators in the diagonal Cartan subalgebra basis Y(a)
  return sp.simplify(sp.conjugate(sp.transpose(modal()))*A(a)*modal())

In [25]:
Y(3)

Matrix([
[0, 0, 0,  0,   0,    0,    0,   0],
[0, 0, 0,  0,   0,    0,    0,   0],
[0, 0, 1,  0,   0,    0,    0,   0],
[0, 0, 0, -1,   0,    0,    0,   0],
[0, 0, 0,  0, 1/2,    0,    0,   0],
[0, 0, 0,  0,   0, -1/2,    0,   0],
[0, 0, 0,  0,   0,    0, -1/2,   0],
[0, 0, 0,  0,   0,    0,    0, 1/2]])

In [26]:
Y(8)

Matrix([
[0, 0, 0, 0,         0,          0,         0,          0],
[0, 0, 0, 0,         0,          0,         0,          0],
[0, 0, 0, 0,         0,          0,         0,          0],
[0, 0, 0, 0,         0,          0,         0,          0],
[0, 0, 0, 0, sqrt(3)/2,          0,         0,          0],
[0, 0, 0, 0,         0, -sqrt(3)/2,         0,          0],
[0, 0, 0, 0,         0,          0, sqrt(3)/2,          0],
[0, 0, 0, 0,         0,          0,         0, -sqrt(3)/2]])

Eigenvectors of the original adjoint basis

In [27]:
def Es(a):
  # Eigenvectors of A(a) |E_a>=S|A_a>
  return sp.simplify(modal()*As(a))

These states reveal the identity of the $SU(3)$ ladder operators

In [28]:
Es(1)==As(3)

True

In [29]:
Es(2)==As(8)

True

In [30]:
Es(3)==(As(1)+sp.I*As(2))/sp.sqrt(2)

True

In [31]:
Es(4)==(As(1)-sp.I*As(2))/sp.sqrt(2)

True

In [32]:
Es(5)==(As(4)+sp.I*As(5))/sp.sqrt(2)

True

In [33]:
Es(6)==(As(4)-sp.I*As(5))/sp.sqrt(2)

True

In [34]:
Es(7)==(As(6)+sp.I*As(7))/sp.sqrt(2)

True

In [35]:
Es(8)==(As(6)-sp.I*As(7))/sp.sqrt(2)

True

Cartan Basis Adjoint Representation

In [36]:
#Adjoint Generators in Cartan basis
H1a=Y(3)
H2a=Y(8)
#E_{\alpha_{1+2}}
Ep12a=(Y(1)+sp.I*Y(2))/sp.sqrt(2)
#E_{-\alpha_{1+2}}
Em12a=(Y(1)-sp.I*Y(2))/sp.sqrt(2)
#E_{\alpha_{1}}
Ep1a=(Y(4)+sp.I*Y(5))/sp.sqrt(2)
#E_{-\alpha_{1}}
Em1a=(Y(4)-sp.I*Y(5))/sp.sqrt(2)
#E_{\alpha_{2}}
Ep2a=(Y(6)-sp.I*Y(7))/sp.sqrt(2)
#E_{-\alpha_{2}}
Em2a=(Y(6)+sp.I*Y(7))/sp.sqrt(2)

In [37]:
Ep12a*As(7)

Matrix([
[        0],
[        0],
[        0],
[        0],
[sqrt(2)/2],
[        0],
[        0],
[        0]])

In [38]:
H1a*As(5)

Matrix([
[  0],
[  0],
[  0],
[  0],
[1/2],
[  0],
[  0],
[  0]])

In [39]:
H2a*As(5)

Matrix([
[        0],
[        0],
[        0],
[        0],
[sqrt(3)/2],
[        0],
[        0],
[        0]])

Cartan Basis Fundamental Representation

In [41]:
#Fundamental Generators in Cartan basis
H1=T(3)
H2=T(8)
#E_{\alpha_{1+2}}
Ep12=(T(1)+sp.I*T(2))/sp.sqrt(2)
#E_{-\alpha_{1+2}}
Em12=(T(1)-sp.I*T(2))/sp.sqrt(2)
#E_{\alpha_{1}}
Ep1=(T(4)+sp.I*T(5))/sp.sqrt(2)
#E_{-\alpha_{1}}
Em1=(T(4)-sp.I*T(5))/sp.sqrt(2)
#E_{\alpha_{2}}
Ep2=(T(6)-sp.I*T(7))/sp.sqrt(2)
#E_{-\alpha_{2}}
Em2=(T(6)+sp.I*T(7))/sp.sqrt(2)

$SU(3)$ Algebra in Cartan Basis

In [42]:
comm(H1,Ep12)==H1a[2,2]*Ep12

True

In [43]:
comm(H2,Ep12)==H2a[2,2]*Ep12

True

In [44]:
comm(H1,Em12)==H1a[3,3]*Em12

True

In [45]:
comm(H2,Em12)==H2a[3,3]*Em12

True

In [46]:
comm(H1,Ep1)==H1a[4,4]*Ep1

True

In [47]:
comm(H2,Ep1)==H2a[4,4]*Ep1

True

In [48]:
comm(H1,Em1)==H1a[5,5]*Em1

True

In [49]:
comm(H2,Em1)==H2a[5,5]*Em1

True

In [50]:
comm(H1,Ep2)==H1a[7,7]*Ep2

True

In [51]:
comm(H2,Ep2)==H2a[7,7]*Ep2

True

In [52]:
comm(H1,Em2)==H1a[6,6]*Em2

True

In [53]:
comm(H2,Em2)==H2a[6,6]*Em2

True

In [54]:
comm(Ep12,Em12)==H1a[2,2]*H1+H2a[2,2]*H2

True

In [55]:
comm(Ep1,Em1)==H1a[4,4]*H1+H2a[4,4]*H2

True

In [56]:
comm(Ep2,Em2)==H1a[7,7]*H1+H2a[7,7]*H2

True

In [57]:
comm(Ep12,Em1)==-Ep2/sp.sqrt(2)

True

In [58]:
comm(Ep12,Em2)==Ep1/sp.sqrt(2)

True

In [59]:
comm(Ep1,Ep2)==Ep12/sp.sqrt(2)

True

In [60]:
comm(Ep1,Em2)

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [61]:
comm(Ep12,Ep1)

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [62]:
comm(Ep12,Ep2)

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])